**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Dropout,Flatten

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function acts prepares the move to the next state of the system by returning the next action to perform.

The next action to perform is either chosen according to previously defined policy - exploitation step - (by calling the method learned_act) or by exploring a random action with probability epsilon -exploration step-. 

The random exploration performed with probability epsilon allows the agent to deviate from the previously defined policy thus giving it the possibility to improve its performance.

Once a policy with high Q-value has been found, the method gives us the possibility to stop the exploration and only perform exploitation by setting train to False.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        # CODAGE RVB DE CHAQUE POINT, un vecteur de grid_size ligne contenant en entree une matrice de gridsize*3, codage rvb
        #de chaque case de chaque ligne
        b = np.zeros((self.grid_size,self.grid_size,3))+128 #everything starts in grey
        b[self.board>0,0] = 256 # bonus in red
        b[self.board < 0, 2] = 256  #malus in blue
        b[self.x,self.y,:]=256 #position cursor in white
        b[-2:,:,:]=0  # border in black
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 #modification made here

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3: #extremité droite de la grille
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1: 
            if self.x == 2: #extrémité gauche de la grille, l'agent réalise la seule action possible en x quand il tire 01
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0 #must be set to zero, because cheese eaten disappears
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:] # vision de 2 cases

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2) #random position of cheeses
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2) #random position of traps
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 #modif here
        self.board[self.x,self.y] = 0  #no starting bonus or malus
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        # gives new state of board along with previous position array

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        #we visualise new state around the new position
        #the new position corresponds to the middle array on middle line and right ie see examples below
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train= 10 # set small when debugging
epochs_test= 10# set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

def display2(name):
    #we get an html error with previous function so we define another one to read video
    # Create a VideoCapture object and read from input file 
    cap = cv2.VideoCapture('random0.mp4') 

    # Check if camera opened successfully 
    if (cap.isOpened()== False):  
      print("Error opening video  file") 

    # Read until video is completed 
    while(cap.isOpened()): 

      # Capture frame-by-frame 
      ret, frame = cap.read() 
      if ret == True: 

        # Display the resulting frame 
        cv2.imshow('Frame', frame) 

        # Press Q on keyboard to  exit 
        if cv2.waitKey(25) & 0xFF == ord('q'): 
          break

      # Break the loop 
      else:  
        break

    # When everything done, release  
    # the video capture object 
    cap.release() 

    # Closes all the frames 
    cv2.destroyAllWindows() 
    return

    

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The board array describes the entire state of the N*N board, it keeps track of where the bonuses (cheese) are located and where  the traps are on the board (-1). All other positions with neither trap nor reward are marked with 0. To sum up the board array keeps track of the rewards on each square of the board. It gets updated (with 0) by the method act as the rewards are taken.

The position array is just a representation of where the rat is in the board and the invalid positions.  Since the grid is of length N+4, the first and last two columns and lines are invalid positions for the rat and are thus marked with -1. The square on which the rat stands is set at 1, the invalid frontier values of the board are set at -1 and the rest is set at 0. After act is called position becomes where the rat was before moving. 

position[i, j] indicates whether the agent is at $(i,j)$ or not.

board[i, j] indicates the state of the board at $(i,j)$ - whether there is cheese, poison, or nothing.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        action = np.random.randint(0,4)
        if action==0:
            if s[3,2,1]==-1:
                return 1
            return 0
        elif action ==1:
            if s[1,2,1]==-1:
                return 0
            return 1
        elif action ==2:
            if s[2,3,1]==-1:
                return 3
            return 2
        elif action ==3:
            if s[2,1,1]==-1:
                return 2
            return 3
            
        

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix='random'):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
             # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            #loss = agent.reinforce(prev_state, state,  action, reward, game_over)

   
        
            ##### FILL IN HERE
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))
    return

#### Answer
Here we play with the random agent.

We compute the average score on the elapsed epochs as we test the model and go through several test epochs.

We see that is between -5 and -4 for the random agent.

In [46]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()
epochs_test = 50
test(agent,env,epochs_test,prefix='random')
#HTML(display_videos('random0.mp4'))
display2('random0.mp4')

Win/lose count 9.5/19.0. Average score (-9.5)
Win/lose count 9.5/12.0. Average score (-6.0)
Win/lose count 6.5/12.0. Average score (-5.833333333333333)
Win/lose count 3.0/16.0. Average score (-7.625)
Win/lose count 8.0/11.0. Average score (-6.7)
Win/lose count 8.0/9.0. Average score (-5.75)
Win/lose count 14.0/17.0. Average score (-5.357142857142857)
Win/lose count 11.0/17.0. Average score (-5.4375)
Win/lose count 9.0/10.0. Average score (-4.944444444444445)
Win/lose count 8.0/13.0. Average score (-4.95)
Win/lose count 13.0/18.0. Average score (-4.954545454545454)
Win/lose count 10.0/11.0. Average score (-4.625)
Win/lose count 10.0/15.0. Average score (-4.653846153846154)
Win/lose count 12.5/11.0. Average score (-4.214285714285714)
Win/lose count 9.5/16.0. Average score (-4.366666666666666)
Win/lose count 12.0/18.0. Average score (-4.46875)
Win/lose count 9.0/18.0. Average score (-4.735294117647059)
Win/lose count 10.0/14.0. Average score (-4.694444444444445)
Win/lose count 5.5/10.0. A

In [38]:
display2('random9.mp4')

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




### Answer

$$
\begin{aligned}
    Q^\pi(s,a)= r(s,a) + \gamma \mathbb E_{p^\pi}\left[
        \sum_{t=1} \gamma^{t-1}r(s_t,a_t) | s_1=s, a_1=a
    \right]  \\
    Q^\pi(s,a)= r(s,a) + \gamma \mathbb E_{(s',a')\sim p(\cdot|s,a)}\left[
        \mathbb E_{p^\pi}\left[
        \sum_{t=0} \gamma^{t}r(s_t,a_t) | s_0=s', a_0=a'
        \right]
    \right]  \\
    Q^\pi(s,a)= r(s,a) + \gamma \mathbb E_{(s',a')\sim p(\cdot|s,a)}\left[
        Q^\pi(s',a')
    \right]
\end{aligned}
$$
From which we find:$$
\begin{aligned}
    Q^*(s,a)= \max_\pi Q^\pi(s,a) = Q^{\pi^*}(s,a) \\
    Q^*(s,a)= r(s,a) + \gamma \mathbb E_{(s',a')\sim p^{\pi^*}(s,a)}[
        Q^{\pi^*}(s',a')
    ]
\end{aligned}
$$and because $\pi^*(s) = \arg\max_a Q^*(s,a)$ for a given state $s$. 

Then: $$
\begin{aligned}
    Q^*(s,a) = r(s,a) + \gamma \mathbb E_{s'\sim p(\cdot|s,a)}
    \left[
        \max_{a'} Q^*(s',a')
    \right]
\end{aligned}
$$

Hence we can use as objective function for evaluating a policy, the Bellman error between the two terms of the equality.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [8]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)<=self.max_memory:
            self.memory.append(m)
        else:
            # if max_memory is reached, we forget oldest action and append newest
            self.memory = self.memory[1:]
            self.memory.append(m)
        return
        

    def random_access(self):
        #return random access, and index
        a = np.random.randint(0,len(self.memory))
        return self.memory[a]

***
The pipeline we will use for training is given below:

In [65]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        print( 'epoch {0}/{1}'.format(e,epoch))
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({}) | Average score til now : {}"
              .format(e, epoch, loss, win, lose, win-lose,score/(e+1)))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [118]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
        
        #Add a model variable
        self.model = None
        

    def learned_act(self, s):
        # learned act returns action with maximum q value
        action_values = self.model.predict(s.reshape((1,5,5,self.n_state))) #is action value a list of arrays?
        action = action_values[0].argsort()[-1] 
        return action

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4)) #4 possible actions on each state
        
        ######## FILL IN
        
        #sample minibatch
        for i in range(self.batch_size):
            access = self.memory.random_access()
            state = access[0]
            action = access[2]
            new_state = access[1]
            reward = access[3]
            game_over = access[-1]
            input_states[i] = state #dimension problem?
            if game_over == True: 
                target_q[i,action] = reward #replace action by : if doesn't work
            else:
                target_q[i,action] = reward + self.model.predict(new_state.reshape((1,5,5,self.n_state))).max()*self.discount
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        #on update on weight
        
        target_q = np.clip(target_q, -3, 3)
        
        
        l = self.model.train_on_batch(input_states, target_q) #returns update of the training loss of the model
            
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        # NN Model
     
        
        model = Sequential([
            keras.layers.Flatten(input_shape=(5, 5, self.n_state)),
            Dense(32, activation='relu'),
            Dense(16, activation='relu'),
            Dense(self.n_action)  # output layer
        ])
        
        model.summary()
        
        model.compile(keras.optimizers.adam(lr=lr), "mse")
        self.model = model
        

#### BEST MODEL

we find that good results are achieved with a memory size of 2000, a batch size of 32 with a learning rate of 1e-4

The 'average score till now' is the sum of all scores observed so far divided by the number of elapsed epochs.

The convergence is much slower here than with the CNN the network starts getting good results after 40 epochs of training. We then reach an average score over 2 on 120 epochs and which keeps improving.

Our average score while training is around -0.7 whereas the random agent has an average score of -4.5.

In [120]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=0.0001, epsilon = 0.1, memory_size=2000, batch_size = 32)
epochs_train=120
train(agent, env, epochs_train, prefix='fc_train')
#HTML(display_videos('fc_train10.mp4'))

Model: "sequential_60"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_60 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_230 (Dense)            (None, 32)                1632      
_________________________________________________________________
dense_231 (Dense)            (None, 16)                528       
_________________________________________________________________
dense_232 (Dense)            (None, 4)                 68        
Total params: 2,228
Trainable params: 2,228
Non-trainable params: 0
_________________________________________________________________
epoch 0/120
Epoch 000/120 | Loss 0.0042 | Win/lose count 1.0/2.0 (-1.0) | Average score til now : -1.0
epoch 1/120
Epoch 001/120 | Loss 0.0126 | Win/lose count 6.5/8.0 (-1.5) | Average score til now : -1.25
epoch 2/120
Epoch 002/120 | Loss 0.0056 | 

Epoch 064/120 | Loss 0.0132 | Win/lose count 7.5/7.0 (0.5) | Average score til now : 0.3384615384615385
epoch 65/120
Epoch 065/120 | Loss 0.0161 | Win/lose count 7.0/3.0 (4.0) | Average score til now : 0.3939393939393939
epoch 66/120
Epoch 066/120 | Loss 0.0056 | Win/lose count 5.5/2.0 (3.5) | Average score til now : 0.44029850746268656
epoch 67/120
Epoch 067/120 | Loss 0.0138 | Win/lose count 10.0/4.0 (6.0) | Average score til now : 0.5220588235294118
epoch 68/120
Epoch 068/120 | Loss 0.0215 | Win/lose count 6.5/5.0 (1.5) | Average score til now : 0.5362318840579711
epoch 69/120
Epoch 069/120 | Loss 0.0167 | Win/lose count 7.5/7.0 (0.5) | Average score til now : 0.5357142857142857
epoch 70/120
Epoch 070/120 | Loss 0.0120 | Win/lose count 7.5/2.0 (5.5) | Average score til now : 0.6056338028169014
epoch 71/120
Epoch 071/120 | Loss 0.0103 | Win/lose count 9.0/6.0 (3.0) | Average score til now : 0.6388888888888888
epoch 72/120
Epoch 072/120 | Loss 0.0095 | Win/lose count 8.5/1.0 (7.5) | A

In [28]:
display2('fc_train99.mp4')

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [124]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
    
        model = Sequential([
            Conv2D(32, 3, input_shape=(5, 5, self.n_state), activation='relu', padding="same"),
            Conv2D(64, 3, activation='relu', padding="same"),
            Conv2D(32, 3, activation='relu', padding="same"),
            keras.layers.Flatten(),
            Dense(self.n_action)  # output
        ])
        
        model.summary()
        
        model.compile(keras.optimizers.adam(lr=lr), "mse")
        self.model = model

#### Answer
Likewise we achieve satisfactory results with the following configuration ie: lr=.00001, epsilon = 0.1, memory_size=2000, batch_size = 32

The average score seems to be slightly higher around 3.5 after a few training epochs. The convergence seems to happen quite quickly.

In [117]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.00001, epsilon = 0.1, memory_size=2000, batch_size = 32)
epochs_train = 85
train(agent,env,epochs_train,prefix='cnn_train')
#HTML(display_videos('cnn_train10.mp4'))

Model: "sequential_58"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_58 (Conv2D)           (None, 5, 5, 32)          608       
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 5, 5, 64)          18496     
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 5, 5, 32)          18464     
_________________________________________________________________
flatten_58 (Flatten)         (None, 800)               0         
_________________________________________________________________
dense_226 (Dense)            (None, 4)                 3204      
Total params: 40,772
Trainable params: 40,772
Non-trainable params: 0
_________________________________________________________________
epoch 0/85
Epoch 000/085 | Loss 0.0114 | Win/lose count 3.5/2.0 (1.5) | Average score til now : 1.5
epoch 1/85
Epoc

Epoch 064/085 | Loss 0.0125 | Win/lose count 11.5/5.0 (6.5) | Average score til now : 4.046153846153846
epoch 65/85
Epoch 065/085 | Loss 0.0098 | Win/lose count 7.0/7.0 (0.0) | Average score til now : 3.984848484848485
epoch 66/85
Epoch 066/085 | Loss 0.0197 | Win/lose count 4.5/3.0 (1.5) | Average score til now : 3.9477611940298507
epoch 67/85
Epoch 067/085 | Loss 0.0124 | Win/lose count 7.0/5.0 (2.0) | Average score til now : 3.9191176470588234
epoch 68/85
Epoch 068/085 | Loss 0.0122 | Win/lose count 5.5/1.0 (4.5) | Average score til now : 3.927536231884058
epoch 69/85
Epoch 069/085 | Loss 0.0082 | Win/lose count 9.0/3.0 (6.0) | Average score til now : 3.9571428571428573
epoch 70/85
Epoch 070/085 | Loss 0.0125 | Win/lose count 3.5/4.0 (-0.5) | Average score til now : 3.8943661971830985
epoch 71/85
Epoch 071/085 | Loss 0.0123 | Win/lose count 1.0/2.0 (-1.0) | Average score til now : 3.826388888888889
epoch 72/85
Epoch 072/085 | Loss 0.0090 | Win/lose count 9.5/4.0 (5.5) | Average scor

In [146]:
display2('cnn_train20.mp4')

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

We load both models trained for T=0.3. We see that the performances of the CNN are a lot better with average score much greater than  that of the fully connected network.

However the display of the trajectories show that the agent tends to stay on only one side of the board, especially for low temperatures.


This results in sub-optimal performance for certain temperatures. For instance with  T=0.8 there are many rewards to collect and we should expect a really high score (>15 with the random model) but we only see a score of 4.

For T=0.1 the performance is on par with T=0.3.

The CNN seems to be quite good at avoiding losses but this is in part due to a low exploration of the board, this also leads to a lower average number of wins than the random explorer albeit with a better overall score, for high temperatures like T=0.8.

The FC is suboptimal for high temperatures like 0.8 with an average score around 9 whereas the random explorer achieves a score of 15. This is due to low exploration in an environment where there are many rewards to reap.



In [125]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=1e-5, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=1e-4, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
epochs_test = 50
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Model: "sequential_61"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_61 (Conv2D)           (None, 5, 5, 32)          608       
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 5, 5, 64)          18496     
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 5, 5, 32)          18464     
_________________________________________________________________
flatten_61 (Flatten)         (None, 800)               0         
_________________________________________________________________
dense_233 (Dense)            (None, 4)                 3204      
Total params: 40,772
Trainable params: 40,772
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_62"
_________________________________________________________________
Layer (type)              

In [126]:
epochs_test = 50
env = Environment(grid_size=size, max_time=T,temperature=0.1)
print('Test of the CNN with Temperature 0.1')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC with Temperature 0.1')
test(agent_fc,env,epochs_test,prefix='fc_test')


Test of the CNN with Temperature 0.1
Win/lose count 6.0/3.0. Average score (3.0)
Win/lose count 5.5/1.0. Average score (3.75)
Win/lose count 7.5/3.0. Average score (4.0)
Win/lose count 4.0/1.0. Average score (3.75)
Win/lose count 3.0/0. Average score (3.6)
Win/lose count 5.5/7.0. Average score (2.75)
Win/lose count 6.0/0. Average score (3.2142857142857144)
Win/lose count 2.0/1.0. Average score (2.9375)
Win/lose count 5.0/3.0. Average score (2.8333333333333335)
Win/lose count 4.0/2.0. Average score (2.75)
Win/lose count 10.5/5.0. Average score (3.0)
Win/lose count 3.0/1.0. Average score (2.9166666666666665)
Win/lose count 3.5/4.0. Average score (2.6538461538461537)
Win/lose count 10.0/1.0. Average score (3.107142857142857)
Win/lose count 4.0/0. Average score (3.1666666666666665)
Win/lose count 7.5/6.0. Average score (3.0625)
Win/lose count 6.5/9.0. Average score (2.735294117647059)
Win/lose count 4.0/2.0. Average score (2.6944444444444446)
Win/lose count 2.0/4.0. Average score (2.447368

In [127]:
env = Environment(grid_size=size, max_time=T,temperature=0.8)
print('Test of the CNN with Temperature 0.8')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC with Temperature 0.8')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN with Temperature 0.8
Win/lose count 14.5/1.0. Average score (13.5)
Win/lose count 21.5/4.0. Average score (15.5)
Win/lose count 15.0/2.0. Average score (14.666666666666666)
Win/lose count 22.0/2.0. Average score (16.0)
Win/lose count 29.0/4.0. Average score (17.8)
Win/lose count 24.0/4.0. Average score (18.166666666666668)
Win/lose count 40.5/7.0. Average score (20.357142857142858)
Win/lose count 24.0/2.0. Average score (20.5625)
Win/lose count 25.5/4.0. Average score (20.666666666666668)
Win/lose count 18.5/1.0. Average score (20.35)
Win/lose count 19.5/5.0. Average score (19.818181818181817)
Win/lose count 17.0/2.0. Average score (19.416666666666668)
Win/lose count 31.0/3.0. Average score (20.076923076923077)
Win/lose count 23.5/2.0. Average score (20.178571428571427)
Win/lose count 14.5/3.0. Average score (19.6)
Win/lose count 36.0/3.0. Average score (20.4375)
Win/lose count 13.5/2.0. Average score (19.91176470588235)
Win/lose count 17.5/1.0. Average score (19.722222

In [ ]:
HTML(display_videos('cnn_test10.mp4'))

In [ ]:
HTML(display_videos('fc_test10.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



The results are a lot better and their performance relative to the random agent no longer suffers depending on the temperature. For T=0.8 we collect an enormous amount of reward (39 vs 19 on average when we do not memorize visited positions). The video shows that adding malus on visited positions indeed forces the agent to explore new areas.
However, the downside is that the agent seems to not go back to areas explored where there are still rewards remaining.

In [129]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        print( 'epoch {0}/{1}'.format(e,epoch))
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action,train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({}) | Average score til now : {}"
              .format(e, epoch, loss, win, lose, win-lose,score/(e+1)))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(Environment):
    def __init__(self, *args,**kwargs):
        super(EnvironmentExploring, self).__init__( *args,**kwargs)
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
    
    def act(self, action,train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))
        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 #modification made here

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3: #extremité droite de la grille
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1: 
            if self.x == 2: #extrémité gauche de la grille, l'agent réalise la seule action possible en x quand il tire 01
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        # take care of the malus for visited positions
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward += self.board[self.x, self.y]
        self.board[self.x, self.y] = 0 #must be set to zero, because cheese eaten disappears
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:] # vision de 2 cases

        return state, reward, game_over
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2) #random position of cheeses
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2) #random position of traps
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 #modif here
        self.board[self.x,self.y] = 0  #no starting bonus or malus
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        # gives new state of board along with previous position array

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        #we visualise new state around the new position
        #the new position corresponds to the middle array on middle line and right ie see examples below
        return state
    

    

## use those samples of code:
#In train explore:
#state, reward, game_over = env.act(action, train=True)


'\n## In Environment exploring:\n# You will have to change n_state to 3 because you will use one more layer!\nreward = 0\nif train:\n    reward = -self.malus_position[self.x, self.y]\nself.malus_position[self.x, self.y] = 0.1\n\nreward = reward + self.board[self.x, self.y]\n# 3 "feature" states instead of 2\nstate = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),\n                                self.board.reshape(self.grid_size, self.grid_size,1),\n                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)\n'

In [132]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.00001, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 100, prefix='cnn_train_explore')
#HTML(display_videos('cnn_train_explore10.mp4'))

Model: "sequential_65"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_70 (Conv2D)           (None, 5, 5, 32)          896       
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 5, 5, 64)          18496     
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 5, 5, 32)          18464     
_________________________________________________________________
flatten_65 (Flatten)         (None, 800)               0         
_________________________________________________________________
dense_239 (Dense)            (None, 4)                 3204      
Total params: 41,060
Trainable params: 41,060
Non-trainable params: 0
_________________________________________________________________
epoch 0/100
Epoch 000/100 | Loss 0.0096 | Win/lose count 5.0/22.700000000000024 (-17.700000000000024) | Average sco

Epoch 049/100 | Loss 0.0121 | Win/lose count 20.0/17.799999999999983 (2.200000000000017) | Average score til now : -4.194
epoch 50/100
Epoch 050/100 | Loss 0.0179 | Win/lose count 21.0/15.899999999999975 (5.1000000000000245) | Average score til now : -4.011764705882352
epoch 51/100
Epoch 051/100 | Loss 0.0065 | Win/lose count 25.0/15.599999999999973 (9.400000000000027) | Average score til now : -3.7538461538461525
epoch 52/100
Epoch 052/100 | Loss 0.0195 | Win/lose count 15.0/12.599999999999971 (2.4000000000000288) | Average score til now : -3.6377358490566016
epoch 53/100
Epoch 053/100 | Loss 0.0111 | Win/lose count 9.0/18.60000000000001 (-9.600000000000009) | Average score til now : -3.7481481481481467
epoch 54/100
Epoch 054/100 | Loss 0.0127 | Win/lose count 21.5/17.900000000000006 (3.5999999999999943) | Average score til now : -3.6145454545454534
epoch 55/100
Epoch 055/100 | Loss 0.0140 | Win/lose count 18.0/14.799999999999976 (3.200000000000024) | Average score til now : -3.492857

In [133]:
# Evaluation
agent_cnn.load(name_weights='cnn_train_exploremodel.h5',name_model='cnn_train_exploremodel.json')
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
test(agent,env,epochs_test,prefix='cnn_train_explore')
#HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 25.0/19.6. Average score (5.399999999999999)
Win/lose count 21.5/17.99999999999999. Average score (4.450000000000005)
Win/lose count 20.5/12.299999999999976. Average score (5.700000000000011)
Win/lose count 22.0/18.0. Average score (5.275000000000009)
Win/lose count 21.5/15.29999999999998. Average score (5.4600000000000115)
Win/lose count 14.5/16.899999999999977. Average score (4.150000000000013)
Win/lose count 29.5/16.099999999999973. Average score (5.471428571428587)
Win/lose count 22.5/17.599999999999984. Average score (5.400000000000015)
Win/lose count 22.0/17.8. Average score (5.26666666666668)
Win/lose count 20.0/12.699999999999978. Average score (5.470000000000015)
Win/lose count 21.5/15.499999999999975. Average score (5.518181818181834)
Win/lose count 26.0/14.399999999999979. Average score (6.025000000000016)
Win/lose count 20.5/17.59999999999999. Average score (5.7846153846154005)
Win/lose count 21.0/15.199999999999976. Average score (5.785714285714302)
Win/lose

In [134]:
# Evaluation
agent_cnn.load(name_weights='cnn_train_exploremodel.h5',name_model='cnn_train_exploremodel.json')
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.8)
test(agent,env,epochs_test,prefix='cnn_train_explore')
#HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 56.0/11.799999999999985. Average score (44.20000000000002)
Win/lose count 41.5/12.499999999999972. Average score (36.60000000000002)
Win/lose count 59.5/9.599999999999985. Average score (41.03333333333335)
Win/lose count 40.0/15.99999999999997. Average score (36.77500000000003)
Win/lose count 60.0/15.799999999999981. Average score (38.260000000000026)
Win/lose count 54.5/13.399999999999975. Average score (38.733333333333356)
Win/lose count 61.0/10.699999999999987. Average score (40.38571428571431)
Win/lose count 48.5/17.09999999999999. Average score (39.262500000000024)
Win/lose count 56.5/15.399999999999979. Average score (39.46666666666669)
Win/lose count 58.0/15.199999999999983. Average score (39.800000000000026)
Win/lose count 61.5/13.399999999999983. Average score (40.554545454545476)
Win/lose count 57.0/17.299999999999997. Average score (40.483333333333356)
Win/lose count 49.0/15.19999999999998. Average score (39.96923076923079)
Win/lose count 58.0/10.1999999999999

In [139]:
display2('cnn_test_explore90.mp4')

***